In [ ]:
import sys
from IPython.display import Image, display
import sympy

sys.path.append('..')   # path for local package
import restflow
from restflow import symvec
from restflow import symtools

# Example 2: Neural Networks

The Wilson-Cowan model is a standard model in neuroscience that describes the evolution of the neural activity field $\phi$; how excitatory and inhibitory neurons interact with each other <sup>1</sup>. In [Tiberi 2022] <sup>2</sup>, they study the model beyond mean-field analysis. They use perturbation theory to express the neural field in a polynomial stochastic PDE similar to the KPZ model. The higher orders of this expansion are assumed to be negligible since the dynamics explore a limited range of the gain function. Based on these, they show that the dynamics evolve according to:
\begin{equation*}
\partial_t\phi = \nabla^2(\kappa \phi + c_2 \phi^2 + c_3 \phi^3) + \eta
\end{equation*}
with non-conserved noise ($\alpha=0$). The field $\phi$ is the neural activity field. The $\nabla^2$ in the front of the RHS denotes that these interactions are described by a diffusive process. The terms with $\kappa$, $c_2$ and $c_3$ come from polynomial expansion up to order 3 of the gain function of the Wilson-Cowan model.

The goal is to study the criticality of the computational features of the system such as the decay of the memory or pattern retrieval. For this, they use Wilson's shell renormalization theory by calculating the flow equations of $\kappa$, $c_2$ and $c_3$.

We follow same procedure as in the KPZ by switching to Fourier space. The only non-zero vertices are:
\begin{equation*}
\cal{v}_2(\bold{q}) = - c_2 \bold{q}^2,
\end{equation*}
and 
\begin{equation*}
\cal{v}_3(\bold{q}) = - c_3 \bold{q}^2
\end{equation*}

This practically means that the graphs can have 2-vertices and 3-vertices. This significantly increases the variety of graphs we have compared to the KPZ example. Below, we use ```restflow`` to calculate all of these graphs.

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions. Remark that we set the external legs with zero because all corrections are of order $q^2$- we do not need graphical corrections of terms that depend on the other external legs.

In [ ]:
# define the model
class Neural_network:
    def __init__(self):
        self.alpha = 0
        self.kap, self.c2, self.c3, self.D = sympy.symbols('kappa c2 c3 D')

    def f(self,k):
        return self.kap*k**2

    def v2(self,k1,k2,q):
        return (-self.c2*q**2,1)

    def v3(self,k1,k2,k3,q):
        return (-self.c3*q**2,1)
# dimension and reduced surface area
d, Kd = sympy.symbols('d K_d')
# symbols for vectors - We set p and r equal to 0
_q, _k, _p, dot_kq, dot_pk, dot_qp = sympy.symbols('q k 0 (k·q) 0 0')
_r, dot_qr, dot_pr, dot_rk = sympy.symbols('0 0 0 0')

ctx = symvec.Context()
ctx.add_dot_product(_q,_k,dot_kq)
ctx.add_dot_product(_q,_p,dot_qp)
ctx.add_dot_product(_p,_k,dot_pk)
ctx.add_dot_product(_q,_r,dot_qr)
ctx.add_dot_product(_p,_r,dot_pr)
ctx.add_dot_product(_r,_k,dot_rk)
# create symbolic wave vectors
k = ctx.vector(_k)
q = ctx.vector(_q)
p = ctx.vector(_p)
r = ctx.vector(_r)

model = Neural_network()

# Propagator Corrections

The only one-loop graphs are the following ones:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_1vertex.jpg')
display(pil_img)

<b><ins>2nd Step:</b></ins> Create the graph using the `plot_graph` script. Convert graph into integral using ```convert``` and calulate the integral using ```integrate```:

In [ ]:
# graph (2c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1],0.0)
v[0].link_vertex(v[2],0.0)
v[2].link_vertex(v[1],0.12)
v[2].add_outgoing(0.0)
g = restflow.Graph(v)
g.label_edges(k,[q])
expr = g.convert(model,[q])
I2c = restflow.integrate([expr],3,k,q)
v = [restflow.Vertex() for i in range(2)]
v[0].link_vertex(v[1],0.0)
v[0].link_vertex(v[1],0.12)
v[0].add_outgoing(0.0)
g = restflow.Graph(v)
g.label_edges(k,[q])
expr = g.convert(model,[q])
I3b = restflow.integrate([expr],3,k,q)


Add the contribution of both diagrams, this is the full graphical correction to the propagator:

In [ ]:
Iprop = sympy.simplify(I2c+I3b)
display(Iprop)

# 2-Vertex Corrections

The only 1-loop diagrams contributing to 2-vertex are:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_2vertex.jpg')
display(pil_img)

We follow the same method for the propagator. The difference is that before we ```integrate```, we use the function ```convert_perm``` because e.g. figures (b) and (d) represent multiple graphs (by permuting the labels of the external legs). 

In [ ]:
# graph (a)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[0].link_vertex(v[2], 0.12)
v[1].link_vertex(v[3], 0.0)
v[2].link_vertex(v[3], 0.0)
v[1].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,q-p])
Ia = restflow.integrate(exprs,3,k,q,p)
# graph (b)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.0)
v[2].add_outgoing(-0.12)
v[3].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,q-p])
Ib = restflow.integrate(exprs,3,k,q,p)
# graph (c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[2].add_outgoing(0.0)
v[2].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,q-p])
Ic = restflow.integrate(exprs,3,k,q,p)
# graph (d)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.0)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,q-p])
Id = restflow.integrate(exprs,3,k,q,p)

We sum the contribution of the integrals for the 2-vertex corrections:

In [ ]:
I2vert = sympy.simplify(Ia+Ib+Ic+Id)
display(I2vert)

# 3-Vertex Corrections

The graphs which contribute to 3-vertex corrections are the following ones:

In [ ]:
pil_img = Image(filename='./figures/modelB_plus_3vertex.jpg')
display(pil_img)

We follow the same procedure as described above: 

In [ ]:
# figure (3c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
v[2].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
I3c = restflow.integrate(exprs,3,k,q,p)
# figure (e)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[0].link_vertex(v[3], 0.12)
v[3].link_vertex(v[2], 0.0)
v[0].add_outgoing(-0.12)
v[1].add_outgoing(-0.12)
v[3].add_outgoing(0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ie = restflow.integrate(exprs,3,k,q,p)
# figure (f)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[0].link_vertex(v[2], 0.12)
v[2].link_vertex(v[3], 0.12)
v[3].link_vertex(v[1], 0.0)
v[0].add_outgoing(-0.12)
v[2].add_outgoing(0.0)
v[3].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
If = restflow.integrate(exprs,3,k,q,p)
# figure (g)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.12)
v[0].link_vertex(v[3], 0.12)
v[3].link_vertex(v[2], 0.0)
v[1].add_outgoing(-0.12)
v[1].add_outgoing(-0.25)
v[3].add_outgoing(0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ig = restflow.integrate(exprs,3,k,q,p)
# figure (h)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.12)
v[3].add_outgoing(0.0)
v[3].add_outgoing(-0.12)
v[2].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ih = restflow.integrate(exprs,3,k,q,p)
# figure (i)
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[1], 0.12)
v[2].add_outgoing(-0.12)
v[2].add_outgoing(0.12)
v[3].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ii = restflow.integrate(exprs,3,k,q,p)
# figure (j)
v = [restflow.Vertex() for i in range(5)]
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[0].link_vertex(v[4], 0.12)
v[4].link_vertex(v[3], 0.0)
v[1].add_outgoing(-0.12)
v[2].add_outgoing(-0.12)
v[4].add_outgoing(0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ij = restflow.integrate(exprs,3,k,q,p)
# figure (k)
v = [restflow.Vertex() for i in range(5)]
v[0].link_vertex(v[1], 0.12)
v[0].link_vertex(v[2], 0.0)
v[2].link_vertex(v[3], 0.0)
v[3].link_vertex(v[4], 0.0)
v[4].link_vertex(v[1], 0.0)
v[2].add_outgoing(-0.12)
v[3].add_outgoing(-0.12)
v[4].add_outgoing(-0.12)
g = restflow.Graph(v)
exprs = g.convert_perm(model,k,[p,r,q-p-r])
Ik = restflow.integrate(exprs,3,k,q,p)

We sum all of the contributions to 3-vertex corrections. The function ```sympy.Poly().as_expr()``` is used to factorize the expression wrt $q^2$:

In [ ]:
I3vert = sympy.Poly(Ie+If+Ig+Ih+Ii+Ij+Ik+I3c,q.sym).as_expr()
display(I3vert)

# Extracting the Corrections of the Model Parameters

We just calculated the corrections to the <b>propagator, 2-vertex and 3-vertex</b>. 
The graphically corrected model parameters $\tilde{x}_i$ are defined wrt the original model parameters $x_i$ as:
\begin{equation*}
\tilde{x}_i=(1+\psi_{x_i}\delta l)x_i.
\end{equation*}
The goal now is to find the corrections $\psi_{x_i}$ with respect to the integrals we just calculated.
For this, we need to:
-   Define the graphically corrected vertices (with graphically corrected parameters)
-   Solve linear equations derived by comparing the coefficients of the monomials of the original vertices with those of the renormalized vertices (see Appendix C of our manuscript)


In [ ]:
# define graphically corrected model parameters
kapt, c1tilda, c2tilda, c3tilda, Dtilda = sympy.symbols('kappat c1t c2t c3t Dt')

# define graphically corrected vertices
def v2tilda(k1,k2,k):
    expr = -c2tilda*k**2
    return sympy.fraction(sympy.cancel(expr))

def v3tilda(k1,k2,k3,k):
    return (-c3tilda*k**2,1)

def compare_coeffs(dict1,dict_original,k,q,p):
    """
    Returns equations between the renormalized model parameters and the original model parameters

    Arguments:
      dict1 (dictionary): dictionary of coefficients of all monomials wrt (k, q, p) of new corrected vertex
      dict_original (dictionary): dictionary of coefficients of all monomials wrt (k, q, p) of integrals contributions
      k, q, p (vectors): wavectors
    Returns:
      equations (array): linear equations to express graphical corrections wrt integral contributions
    """
    eqts = [0]*len(dict1)
    i=0
    for key in dict1:
        eqts[i]= dict1.get(key) - dict_original.get(key)
        i+=1
    eqts = [i for i in eqts if i != 0]
    return eqts

def expand_vertex(expr, q):
    """
    Expands the coefficients of the renormalized 2-vertex

    Arguments:
      expr (symbolic expression): vertex or integral contributions
    Returns:
      dictionary: coefficients of all monomials of multivariate polynomial (k, q, p)
    """
    return symtools.all_coeffs(expr,[q.sym])

def renormalize_vertex(expr,vertex,k,q,p):
    """
    Matches the coefficients of the 2-vertex with the renormalized 2-vertex. Solves the system of linear eqts.

    Arguments:
      expr (symbolic expression): integral contributions
      vertex (symbolic expression): the graphically corrected vertex
      k, q, p (vectors): wavectors
    Returns:
      list: graphical corrections to coefficients of the monomials 
    """
    delta_l = sympy.symbols('δl')

    coeffs = expand_vertex(expr,q)
    coeffs_original = expand_vertex(vertex[0]/vertex[1],q)
    eqts = compare_coeffs(coeffs,coeffs_original,k,q,p)
    if vertex == v2tilda(q,q,q):
        print(eqts)
        return [sympy.simplify(element/(model.c2*delta_l)) for element in list(sympy.linsolve(eqts, [c2tilda]))[0]]
    elif vertex == v3tilda(q,q,q,q):
        return [sympy.simplify(element/(model.c3*delta_l)) for element in list(sympy.linsolve(eqts, [c3tilda]))[0]]

For the $\phi_\kappa$ we use the following relationship:
\begin{equation*}
\frac{1}{\tilde{f}(q)}=\frac{1}{f(q)}+I, 
\end{equation*}
and by taylor expanding, we get:
\begin{equation*}
\tilde{f}(k)\approx f(k)-I, 
\end{equation*}
where $I$ is the integrals we calculated earlier for the propagators.

Using the corrections to the propagators, we get that $\psi_\kappa$ is:

In [ ]:
from IPython.display import Latex
display(Latex('$\psi_\kappa=$'),sympy.simplify(-Iprop/(model.kap*q**2)))

Similarly, we get for the 2-vertex and 3-vertex:

In [ ]:
from IPython.display import Latex
# renormalize c2 parameter
psi_2 = renormalize_vertex(I2vert,v2tilda(q,q,q),k,q,p)[0]
display(Latex('$\psi_2=$'),psi_2)
# renormalize c3 parameter
psi_3 = renormalize_vertex(I3vert,v3tilda(q,q,q,q),k,q,p)[0]
display(Latex('$\psi_3=$'),psi_3)


## Example how to get flow equations from $\psi_i$ corrections

Now that we have calculated the corrections to the propagator, 2-vertex and 3-vertex, we can get the flow equations of the model parameters. 

<b><ins>1st Step</b></ins>: <b>Restore the cut-off of our theory through rescaling $\Lambda/b \rightarrow \Lambda$ </b>. <br> For this, we need to calculate the scaling dimension $\Delta_{x_i}$ of each model parameter $x_i$; this can be found by dimensional analysis (see Section 2.3). Therefore, the renormalized model parameters become: $x_i'=b^{\Delta_{x_i}}\tilde{x}_i$. 

<b><ins>2nd Step</b></ins>: <b>Get Wilson's flow equations for rescaled model parameters</b>. <br> Using the previous equation between $\bar{x}_i$ and $x_i$ and for $b=1+\delta l$, we get:
\begin{equation*}
\partial_l x_i=(\Delta_{x_i}+\psi_{x_i})x_i
\end{equation*}

The problem with this approach is that, in principle, dimensional analysis gives relationships between the scaling dimensions of the model parameters but not exclusively one value. Therefore, the final flow equations are expressed through reduced "dimensionless" model parameters. 

<b><ins>3rd Step</b></ins>: <b>Get flow equations for dimensionless model parameters</b>.  <br>
As an example, from dimensional analysis we get $\Delta_{c_3}+\Delta_D -2 \Delta_\kappa= 2-d$. Therefore we introduce dimensionless variable $\bar{c}_3=\frac{c_3 D}{\kappa^2} K_d \Lambda^{d-2}$. Using chain rule and the flow equations of $c_3$ and $\kappa$ and $D$, we can get the flow equation of $\bar{c}_3$:
\begin{equation*}
\partial_l \bar{c}_3 = \bigg(\frac{\partial_l c_3}{c_3} - 2 \frac{\partial_l \kappa}{\kappa} + \frac{\partial_l D}{D}\bigg) \bar{c}_3 = \bigg(\Delta_{c_3}+\Delta_D-2\Delta_\kappa+\psi_{c_3}-2\psi_{\kappa}\bigg)\bar{c}_3 = \bigg(2-d+\psi_{c_3}-2\psi_{\kappa}\bigg)\bar{c}_3
\end{equation*}

The program calculates the corrections $\psi_{x_i}$ but at this stage it is up to the user to apply the dimensional analysis for the flow equations of the dimensionless parameters.


# Bibliography

<sup>1</sup> Wilson, Hugh R., and Jack D. Cowan. "Excitatory and inhibitory interactions in localized populations of model neurons." Biophysical journal 12.1 (1972): 1-24.

<sup>2</sup> L. Tiberi, J. Stapmanns, T. Kühn, T. Luu, D. Dahmen and M. Helias, Gell-
Mann–Low Criticality in Neural Networks, Phys. Rev. Lett. 128(16), 168301 (2022)